Setting up pyspark enviroment

In [ ]:
# setting up pyspark environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!pip install pyspark
!pip install -U -q PyDrive

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as pyspark_functions
from pyspark import SparkContext, SparkConf


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7b6df3d14384bbceeac30ec062dc8c9b794e87d5610ba37363a466db9f2101b2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# mounted  from drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/in-vehicle-coupon-recommendation.csv", header = True)


In [ ]:
df.show(7)

+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|    destination|passanger|weather|temperature|time|              coupon|expiration|gender|age|    maritalStatus|has_children|           education|occupation|         income| car|  Bar|CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|  Y|
+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|No 

In [ ]:
# select_columns = ["destination" , "passenger", "weather", "temperature", "gender"]
df.show()

+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|    destination|passanger|weather|temperature|time|              coupon|expiration|gender|age|    maritalStatus|has_children|           education|occupation|         income| car|  Bar|CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|  Y|
+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|No 

In [ ]:
columns = ["destination","passanger","weather","temperature","time","coupon"]
filterdata = df.select(*columns)
filterdata.show()

+---------------+---------+-------+-----------+----+--------------------+
|    destination|passanger|weather|temperature|time|              coupon|
+---------------+---------+-------+-----------+----+--------------------+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|     Restaurant(<20)|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House|
|No Urgent Place|Friend(s)|  Sunny|         80| 6PM|     Restaurant(<20)|
|No Urgent Place|Friend(s)|  Sunny|         55| 2PM|Carry out & Take ...|
|No Urgent Place|   Kid(s)|  Sunny|         80|10AM|     Restaurant(<20)|
|No Urgent Place|   Kid(s)|  Sunny|         80|10AM|Carry out & Take ...|
|No Urgent Place|   Kid(s)|  Sunny|         80|10AM|                 Bar|
|No Urgent Place|   Kid(s)|  Sunny|   

In [ ]:
from pyspark.sql.functions import col
filtered_df = df.filter(col("coupon").like("Restaurant"))
filtered_df.show()

+-----------+---------+-------+-----------+----+------+----------+------+---+-------------+------------+---------+----------+------+---+---+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|destination|passanger|weather|temperature|time|coupon|expiration|gender|age|maritalStatus|has_children|education|occupation|income|car|Bar|CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|  Y|
+-----------+---------+-------+-----------+----+------+----------+------+---+-------------+------------+---------+----------+------+---+---+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
+-----------+---------+-------+-----------+----+------+----------+------+---+-------------+------------+---------+----------+------+---+---

In [ ]:
from pyspark.sql.functions import col

# Filter the DataFrame for records where "coupon" indicates a restaurant
restaurant_records = df.filter(col("coupon").like("Restaurant%"))

# Show the filtered restaurant records
restaurant_records.show()


+---------------+---------+-------+-----------+----+-----------------+
|    destination|passanger|weather|temperature|time|           coupon|
+---------------+---------+-------+-----------+----+-----------------+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|  Restaurant(<20)|
|No Urgent Place|Friend(s)|  Sunny|         80| 6PM|  Restaurant(<20)|
|No Urgent Place|   Kid(s)|  Sunny|         80|10AM|  Restaurant(<20)|
|No Urgent Place|   Kid(s)|  Sunny|         80| 2PM|  Restaurant(<20)|
|No Urgent Place|   Kid(s)|  Sunny|         55| 2PM|  Restaurant(<20)|
|           Home|    Alone|  Sunny|         55| 6PM|Restaurant(20-50)|
|           Work|    Alone|  Sunny|         80| 7AM|Restaurant(20-50)|
|           Work|    Alone|  Sunny|         55| 7AM|  Restaurant(<20)|
|No Urgent Place|    Alone|  Sunny|         55| 2PM|  Restaurant(<20)|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|  Restaurant(<20)|
|No Urgent Place|Friend(s)|  Sunny|         80| 6PM|  Restaurant(<20)|
|     

In [ ]:
df.filter((col("passanger")=="Alone")&(col("age") > 21)&(col("coupon").startswith("Restaurant"))).count()

1447

In [ ]:
df.select(col("coupon")).distinct().show()

+--------------------+
|              coupon|
+--------------------+
|     Restaurant(<20)|
|   Restaurant(20-50)|
|                 Bar|
|Carry out & Take ...|
|        Coffee House|
+--------------------+



In [ ]:
from pyspark.sql.functions import col
filtersam = df.filter((col("passanger") == "Alone") &
                      (col("age") > 21) &
                      (col("coupon").startswith("Restaurant")) &
                      (col("maritalStatus") == "Unmarried partner"))

filtersam.show()

+---------------+---------+-------+-----------+----+-----------------+----------+------+---+-----------------+------------+-----------------+--------------------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|    destination|passanger|weather|temperature|time|           coupon|expiration|gender|age|    maritalStatus|has_children|        education|          occupation|         income| car|  Bar|CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|  Y|
+---------------+---------+-------+-----------+----+-----------------+----------+------+---+-----------------+------------+-----------------+--------------------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-----------

In [ ]:
df.show()  #orginal

+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|    destination|passanger|weather|temperature|time|              coupon|expiration|gender|age|    maritalStatus|has_children|           education|occupation|         income| car|  Bar|CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|  Y|
+---------------+---------+-------+-----------+----+--------------------+----------+------+---+-----------------+------------+--------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|No 

In [ ]:
# reading csv file
testdata =spark.read.csv("/content/test data.csv", header=True, inferSchema =True)

In [ ]:
testdata.show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|             gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------

In [ ]:
# rename column
df=testdata.withColumnRenamed("order_number","orderId")
df.show()

+--------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
| orderId|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|             gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+--------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------------------

In [ ]:
testdata.select(col("sales_channel")).distinct().show()

+-------------+
|sales_channel|
+-------------+
|      Express|
|     Showroom|
|   E-Commerce|
| Inside Sales|
+-------------+



In [ ]:
express_sales_df = testdata.filter(testdata["sales_channel"] == "Express")
express_sales_df.show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|             gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------

In [ ]:
from pyspark.sql import functions as F

# Assuming pdata is your DataFrame with sales data
# Filter rows where sales_channel is "Express" and calculate the total gross sales
salesofexpress = testdata.filter(testdata["sales_channel"] == "Express").select(F.sum("gross_sales").alias("total_sales"))

# Show the total gross sales for the "Express" sales channel
salesofexpress.show()


+-----------------+
|      total_sales|
+-----------------+
|205301.9218839499|
+-----------------+



In [ ]:
# concate two columns
from pyspark.sql.functions import concat,lit

# concated two columns and drop
testdata = testdata.withColumn("product",concat(testdata["product_grp_1"],lit(" "), testdata["product_grp_1_desc"])).drop(['product_grp_1_desc'])
testdata.drop('product_grp_1_desc').printSchema()

root
 |-- order_number: integer (nullable = true)
 |-- invoice_number: integer (nullable = true)
 |-- sales_channel: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_type: string (nullable = true)
 |-- order_description: string (nullable = true)
 |-- order_line: integer (nullable = true)
 |-- gl_date: string (nullable = true)
 |-- item_number: string (nullable = true)
 |-- item_description: string (nullable = true)
 |-- price_per_unit: double (nullable = true)
 |-- unit_cost: double (nullable = true)
 |-- gross_sales: double (nullable = true)
 |-- gross_cost: double (nullable = true)
 |-- sales_bonus: double (nullable = true)
 |-- discounts: double (nullable = true)
 |-- company: integer (nullable = true)
 |-- product_grp_2: string (nullable = true)
 |-- product_grp2_desc: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- location: integer (nullable = true)
 |-- location_name: string (nullable = true)
 |-- customer_region: integer (nu

In [ ]:
# drop column
estdata = testdata.drop("product_grp_1_desc")
testdata.show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+--------------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|       product|
+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------

In [ ]:
testdata.show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|             gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+------------+--------------+-------------+-----------+----------+-----------------+----------+--------------------+-----------+--------

In [ ]:
from pyspark.sql import functions as F

testdata.withColumn("gl_date", F.to_date(F.date_format(F.col("gl_date"),"MM-dd-yyyy"))).show()


+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------

In [ ]:
testdata.show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|
+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------

In [ ]:
#concate two columns and name as new column product
testdata.withColumn("product",concat(testdata["product_grp_1"],lit(" "),testdata["product_grp_1_desc"])).show()

# to delete two columns use *
testdata.drop(*('product_grp_1','product_grp_1_desc')).show()

+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+--------------------+--------------+-----------+-----------+-----------+-----------+---------+-------+-------------+------------------+-------------+--------------------+--------+--------+--------------------+---------------+--------------------+--------------------+-------------------------+---------+--------------+
|order_number|invoice_number|sales_channel|customer_id|order_type|order_description|order_line|gl_date|item_number|    item_description|price_per_unit|  unit_cost|gross_sales| gross_cost|sales_bonus|discounts|company|product_grp_1|product_grp_1_desc|product_grp_2|   product_grp2_desc|quantity|location|       location_name|customer_region|customer_region_name|business_unit_region|business_unit_region_name|cust_segm|       product|
+------------+--------------+-------------+-----------+----------+-----------------+----------+-------+-----------+-----------------

In [ ]:

selectcolumns =["order_number","invoice_number","sales_channel","customer_id","order_type","item_description","gross_sales","company"]

df = testdata.select(*selectcolumns)
df.show()

+------------+--------------+-------------+-----------+----------+--------------------+-----------+-------+
|order_number|invoice_number|sales_channel|customer_id|order_type|    item_description|gross_sales|company|
+------------+--------------+-------------+-----------+----------+--------------------+-----------+-------+
|    27115522|      19002665| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|38.41506975|      1|
|    27087599|      18983527| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|167.8675862|      1|
|    26799831|      18788031| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|3.926055395|      1|
|    26763012|      18764418| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|46.38397346|      1|
|    26736947|      18746308| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...| 141.530975|      1|
|    26488557|      18581938| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|23.72961626|      1|
|     3334429|       2272185

In [ ]:
# showing sales of express

from pyspark.sql import functions as F

sales_of_express = df.filter(df["sales_channel"] == "Express") \
                          .select(F.lit("Express").alias("sales_channel"), F.sum("gross_sales").alias("total_sales"))

sales_of_express.show()

+-------------+------------------+
|sales_channel|       total_sales|
+-------------+------------------+
|      Express|473554.78380403225|
+-------------+------------------+



In [ ]:
# # showing sales of express

from pyspark.sql import functions as F

insidesales = df.filter(df["sales_channel"] == "Inside Sales") \
            .select(F.lit("Inside Sales").alias("sales"), F.sum("gross_sales").alias("totalsales"))

# Show the result
insidesales.show()

+------------+-----------------+
|       sales|       totalsales|
+------------+-----------------+
|Inside Sales|866221.1175304711|
+------------+-----------------+



In [ ]:
df.show()

+------------+--------------+-------------+-----------+----------+--------------------+-----------+-------+
|order_number|invoice_number|sales_channel|customer_id|order_type|    item_description|gross_sales|company|
+------------+--------------+-------------+-----------+----------+--------------------+-----------+-------+
|    27115522|      19002665| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|38.41506975|      1|
|    27087599|      18983527| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|167.8675862|      1|
|    26799831|      18788031| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|3.926055395|      1|
|    26763012|      18764418| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|46.38397346|      1|
|    26736947|      18746308| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...| 141.530975|      1|
|    26488557|      18581938| Inside Sales|     101190|        SO|WC-ELEM.STANDAARD...|23.72961626|      1|
|     3334429|       2272185

In [ ]:
# orderby customer id
df.orderBy(col("customer_id").asc()).show()

+------------+--------------+-------------+-----------+----------+--------------------+------------+-------+
|order_number|invoice_number|sales_channel|customer_id|order_type|    item_description| gross_sales|company|
+------------+--------------+-------------+-----------+----------+--------------------+------------+-------+
|    25629751|      18006912|     Showroom|      21121|        C3|DOUCHETHERMSTAAT ...|-26.00616568|      1|
|    26104912|      18325494| Inside Sales|      21131|        SR|KNELSET NIKKEL 1/...| 3.602395035|      1|
|    26104912|      18325494| Inside Sales|      21131|        SR|PERS KOPP.16 MLC ...| 24.92006855|      1|
|    26132919|      18342286|      Express|      21131|        C5|THERMOSTAATKOP DX...|-12.75195651|      1|
|    26104912|      18325494| Inside Sales|      21131|        SR|RAD.KOMPAKT T21 6...| 46.67317578|      1|
|    26104912|      18325494| Inside Sales|      21131|        SR|CV BUIS EV 15X1,2...| 5.415521698|      1|
|    26132919|     

In [ ]:
# showing sales of saleschannel

df.groupby("sales_channel").agg(F.sum("gross_sales").alias("totalsales")).show()

+-------------+------------------+
|sales_channel|        totalsales|
+-------------+------------------+
|      Express|473554.78380403225|
|     Showroom| 160514.0021778071|
|   E-Commerce| 359818.2247454213|
| Inside Sales| 866221.1175304711|
+-------------+------------------+

